In [4]:
import requests
import numpy as np
from LSBSteg import encode
from riddle_solvers import riddle_solvers
import random
base_url = 'http://13.53.169.72:5000/attempts/professional'
team_id = "wI4FoRq"


In [5]:
def init_fox(team_id):
    '''
    In this function you need to hit to the endpoint to start the game as a fox with your team id.
    If a successful response is returned, you will receive back the message that you can break into chunks
    and the carrier image that you will encode the chunks in it.
    '''
    response = requests.post(url=base_url+'/fox/start', json={'teamId': team_id})
    status_code = response.status_code
    data = response.json() if status_code == 200 else {}

    message = data.get('msg')
    image_carrier = data.get('carrier_image')

    print('Status:', status_code)
    print('Data:', data)

    return message, image_carrier

In [ ]:
def get_riddle(team_id, riddle_id):

    # API endpoint URL for requesting a riddle
    url = base_url + '/fox/get-riddle'
    payload = {
        "teamId": team_id,
        "riddleId": riddle_id
    }

    response = requests.post(url, json=payload)

    if response.status_code == 200:
        # Return the response JSON
        return response.json()
    else:
        print(f"Failed to get riddle. Status code: {response.status_code}")
        return None

# # Example
# riddle_id = "sec_hard"
# test_case = get_riddle(team_id, riddle_id)
# print(test_case)


In [ ]:
def solve_riddle(team_id, riddle_id):
    url = base_url+'/fox/solve-riddle'
    riddle_testcase = get_riddle(team_id, riddle_id)
    solver_function = riddle_solvers[riddle_id]
    solution = solver_function(riddle_testcase)

    payload = {
        "teamId": team_id,
        "solution": solution
    }

    # Send POST
    response = requests.post(url, json=payload)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to solve {riddle_id} riddle. Status code: {response.status_code}")
        return None

# # Example usage:
# solution = "solution"
# riddle_solution_response = solve_riddle(team_id, solution)
# print(riddle_solution_response)


In [ ]:

def send_message(team_id, messages, message_entities=['F', 'E', 'R']):
    url = base_url + "/fox/send-message"

    payload = {
        "teamId": team_id,
        "messages": messages,
        "messageEntities": message_entities
    }
    response = requests.post(url, json=payload)

    if response.status_code == 200:
        return response.json()["status"]
    else:
        print(f"Failed to send message. Status code: {response.status_code}")
        return None

# # Example
# team_id = "wI4FoRq"
# messages = [np.array([[0, 0], [0, 0]]), np.array([[1, 1], [1, 1]]), np.array([[2, 2], [2, 2]])]
# message_entities = ['F', 'E', 'R']
# send_message_response = send_message(team_id, messages, message_entities)
# print(send_message_response)


In [ ]:
def generate_message_array(message, image_carrier):  
    '''
    In this function you will need to create your own startegy. That includes:
        1. How you are going to split the real message into chunkcs
        2. Include any fake chunks
        3. Decide what 3 chuncks you will send in each turn in the 3 channels & what is their entities (F,R,E)
        4. Encode each chunck in the image carrier  
    '''

    '''
    our strategy maximizes the score by using 3 chunks, 3 real, 6 fake, 0 empty
    making the fake = 6 maximizes the second parameter,
    making the empty = 0 maximizes the first parameter
    '''
    # split the message into 3 chunks,
    # each one will not contain the same number of characters
    size = len(message)
    chunk1size = random.randint(1, size-2)
    chunk2size = random.randint(1, size-chunk1size-1)
    chunk3size = size - chunk1size - chunk2size
    chunk1 = message[:chunk1size]
    chunk2 = message[chunk1size:chunk1size+chunk2size]
    chunk3 = message[chunk1size+chunk2size:]

    real_chunks = [chunk1, chunk2, chunk3]
    chunk_sizes = [chunk1size, chunk2size, chunk3size]
    fake_chunks = []

    # generate 6 fake chunks which are the same size as the real chunks, each character is a random letter
    for i in range(6):
        fake_chunk = ''.join([chr(random.randint(97, 122)) for _ in range(chunk_sizes[i//2])])
        fake_chunks.append(fake_chunk)
    
    for i in range(3):
        # encode the real chunks into the carrier image
        real_chunk = real_chunks[i]
        real_image = encode(image_carrier, real_chunk)
        real_image = real_image.tolist()
        # encode the 2 fake chunks into the carrier image
        fake_chunk1 = fake_chunks[i]
        fake_chunk2 = fake_chunks[i+1]
        fake_image1 = encode(real_image, fake_chunk1)
        fake_image2 = encode(fake_image1, fake_chunk2)
        fake_image1 = fake_image1.tolist()
        fake_image2 = fake_image2.tolist()

        
        print(f'message {i}:', send_message(team_id, [real_image, fake_image1, fake_image2], ['R', 'F', 'F']))
        


In [ ]:
def end_fox(team_id):
    url = base_url + "/fox/end-game"

    payload = {
        "teamId": team_id
    }

    response = requests.post(url, json=payload)

    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to end game. Status code: {response.status_code}")
        return None

# # Example
# team_id = "wI4FoRq"
# end_game_response = end_fox(team_id)
# print(end_game_response)


In [ ]:
def submit_fox_attempt(team_id):

    try:
        # Step 1: Initialize the game as fox
        secret_msg, carrier = init_fox(team_id)

        # Step 2: Solve riddles
        # 'cv_easy', 'cv_medium', 'ml_easy', 'ml_medium', 'sec_hard'
        riddle_ids = ['problem_solving_easy', 'problem_solving_medium', 'problem_solving_hard']
        for riddle_id in riddle_ids:
            solve_riddle(team_id, riddle_id)

        # Step 3&4: Strategy & Send the messages
        # Call the send_message function with your messages and their entities
        generate_message_array(secret_msg, carrier)

        # Step 5: End the game
        end_game_response = end_fox(team_id)
        return end_game_response

    except Exception as e:
        print(f"Error submitting fox attempt: {e}")
        return None
